# Databases Overview

CIMantic Graphs provides a unified interface for accessing CIM power system models stored in multiple database backends and file formats. This flexibility allows users to choose the best storage solution for their specific use case, from simple XML files for small test cases to enterprise-grade graph databases for utility-scale networks.

## Available Database Backends

CIMantic Graphs supports the following database connections:

### File-Based Parsers

* **XMLFile** - CIM XML/RDF file parser (IEC 61970-301 compliant)
  - Best for: Small test cases (IEEE 13, 123-bus)
  - No database infrastructure required
  - Documentation: [XML File Parser](3_8_xml_file_parser.ipynb)

* **JSON-LD File Parser** - JSON-LD file parser
  - Best for: Web-based applications and REST APIs
  - Modern JSON format with RDF semantics
  - Documentation: [JSON-LD Parser](3_9_jsonld_file_parser.ipynb)

### SPARQL Triple Stores

* **Blazegraph** - High-performance RDF database
  - Best for: Large distribution models (IEEE 8500+)
  - Fast query execution and bulk loading
  - Documentation: [Blazegraph](3_3_blazegraph.ipynb)

* **GraphDB** - Enterprise semantic graph database (Ontotext)
  - Best for: Enterprise deployments with reasoning requirements
  - Advanced SPARQL features and reasoning engines
  - Documentation: [GraphDB](3_5_graphdb.ipynb)

* **RDFlib** - Pure Python RDF library with in-memory and persistent stores
  - Best for: Python-native applications and testing
  - No external database required
  - Documentation: [RDFlib](3_10_rdflib_parser.ipynb)

### Labeled Property Graph Databases

* **Neo4j** - Leading property graph database
  - Best for: Graph algorithms and path analysis
  - Cypher query language
  - Documentation: [Neo4j](3_4_neo4j.ipynb)

### Specialized Connections

* **GridAPPS-D** - Integration with GridAPPS-D platform
  - Best for: Distribution grid simulation workflows
  - Direct integration with GOSS message bus
  - Documentation: [GridAPPS-D](3_7_gridappsd.ipynb)

* **MySQL** - Relational database with JSON-LD storage
  - Best for: Integration with existing SQL infrastructure
  - Experimental support
  - Documentation: [MySQL](3_6_mysql.ipynb)

----

## Architecture

All database connections in CIMantic Graphs implement the `ConnectionInterface` abstract base class, which defines a consistent API for:

* **Connection Management**: `connect()`, `disconnect()`
* **Query Execution**: `execute()`, `update()`
* **Object Retrieval**: `get_object()`, `get_from_triple()`
* **Graph Creation**: `create_new_graph()`, `create_distributed_graph()`
* **Graph Expansion**: `get_all_edges()`, `get_all_attributes()`
* **Data Upload**: `upload()`

This abstraction allows users to switch between database backends without changing their application code.

### ConnectionInterface Class Diagram

In [ ]:
from mermaid import Mermaid

diagram_text = """%%{init: {"theme":"base"}}%%
classDiagram
    class ConnectionInterface {
        <<abstract>>
        +Graph: TypeAlias
        +cim_profile: str
        +namespace: str
        +iec61970_301: str
        +log_level: int
        +connect()*
        +disconnect()*
        +execute(query)* QueryResponse
        +get_object(mRID, graph) object
        +get_from_triple(subject, predicate, graph) list
        +create_new_graph(container, graph) Graph
        +create_distributed_graph(area, graph) Graph
        +get_all_edges(graph, cim_class)*
        +get_all_attributes(graph, cim_class)*
        +upload(graph)*
        #create_object(graph, class_type, uri) object
        #create_edge(graph, cim_class, id, attr, edge_class, edge_mRID) object
        #create_value(graph, cim_class, id, attr, value) value
        #check_attribute(cim_class, attribute) str
    }
    
    class SPARQLEndpointConnection {
        <<abstract>>
        +connection_obj: object
        #_setup_connection()*
        #_execute_raw_query(query)* QueryResponse
        #_parse_result_field(result, field)* str
        #_update_raw(update)* str
        #_get_namespaces() list~str~
        +parse_node_query(graph, output) Graph
        +edge_query_parser(output, graph, class) list
        +build_graph_from_list(graph, mrid_list) Graph
    }
    
    class XMLFile {
        +filename: str
        +tree: ElementTree
        +root: Element
        +namespaces: dict
        +parse_nodes(element) Identity
        +parse_edges(element)
        +parse_value(sub_element, class, id)
        +extract_namespaces_from_header() dict
    }
    
    class BlazegraphConnection {
        +url: str
        +namespaces: list~str~
    }
    
    class GraphDBConnection {
        +url: str
        +repository: str
    }
    
    class RDFlibConnection {
        +store_type: str
        +identifier: str
    }
    
    class Neo4jConnection {
        +host: str
        +port: int
        +database: str
    }
    
    ConnectionInterface <|-- SPARQLEndpointConnection
    ConnectionInterface <|-- XMLFile
    SPARQLEndpointConnection <|-- BlazegraphConnection
    SPARQLEndpointConnection <|-- GraphDBConnection
    SPARQLEndpointConnection <|-- RDFlibConnection
    ConnectionInterface <|-- Neo4jConnection
"""

Mermaid(diagram_text)

----

## SPARQL Endpoint Architecture

The majority of database backends (Blazegraph, GraphDB, RDFlib) share a common SPARQL-based implementation through the `SPARQLEndpointConnection` abstract class. This unified architecture provides:

* **Common query generation** via the `cimgraph.queries.sparql` module
* **Shared parsing logic** for SPARQL query results
* **Parallel query execution** with batching for performance
* **Template method pattern** where subclasses implement only database-specific connection details

### Database-Specific Implementation Requirements

Each SPARQL database backend must implement four abstract methods:

1. **`_setup_connection()`** - Initialize the database-specific connection object
2. **`_execute_raw_query(query_message)`** - Execute a SPARQL query and return raw results
3. **`_parse_result_field(result, field_name)`** - Extract field values from query results
4. **`_update_raw(update_message)`** - Execute a SPARQL update statement

### Optional Customization

Subclasses may optionally override:

* **`_get_namespaces()`** - Return list of namespaces for enumeration parsing (e.g., Blazegraph supports multiple namespaces)

----

## Common SPARQL Endpoint Methods

**Note:** This section documents the methods shared by all SPARQL-based database backends (Blazegraph, GraphDB, RDFlib). Individual database documentation will reference this section rather than duplicating the content.

The following methods are implemented in `SPARQLEndpointConnection` and available to all SPARQL databases.

### Connection Management

#### connect()

Establishes a connection to the SPARQL endpoint.

**Parameters:** None

**Returns:** None

**Behavior:**
* Checks if `connection_obj` is already initialized
* If not, calls `_setup_connection()` to create database-specific connection
* Called automatically by query methods if connection not established

**Usage:**
```python
from cimgraph.databases import BlazegraphConnection
db = BlazegraphConnection()
db.connect()  # Usually called automatically
```

**Source:** `cimgraph/databases/sparql_endpoint.py:118`

#### disconnect()

Disconnects from the SPARQL endpoint by releasing the connection object.

**Parameters:** None

**Returns:** None

**Behavior:**
* Sets `connection_obj` to `None`
* Releases database connection resources

**Usage:**
```python
db.disconnect()
```

**Source:** `cimgraph/databases/sparql_endpoint.py:123`

### Query Execution

#### execute(query_message)

Executes a SPARQL SELECT query on the endpoint.

**Parameters:**
* `query_message` (str): The SPARQL query string to execute

**Returns:**
* `QueryResponse`: Dictionary containing query results in SPARQL JSON format

**Behavior:**
* Ensures connection is established via `connect()`
* Delegates to database-specific `_execute_raw_query()`
* Returns results in standard SPARQL JSON format

**Usage:**
```python
query = "SELECT ?s ?p ?o WHERE { ?s ?p ?o } LIMIT 10"
results = db.execute(query)
```

**Source:** `cimgraph/databases/sparql_endpoint.py:131`

#### update(update_message)

Executes a SPARQL UPDATE statement (INSERT, DELETE, etc.).

**Parameters:**
* `update_message` (str): The SPARQL update statement to execute

**Returns:**
* `str`: Response message from the database

**Behavior:**
* Ensures connection is established via `connect()`
* Delegates to database-specific `_update_raw()`
* Used internally by `upload()` method

**Usage:**
```python
update = "DELETE { ?s ?p ?o } WHERE { ?s ?p ?o }"
response = db.update(update)
```

**Source:** `cimgraph/databases/sparql_endpoint.py:145`

### Object Retrieval

#### get_object(mRID, graph=None)

Retrieves a single CIM object from the database by its mRID.

**Parameters:**
* `mRID` (str): The master resource identifier (UUID) of the object
* `graph` (dict): Optional existing graph (creates new if `None`)

**Returns:**
* `object`: The retrieved CIM object, or `None` if not found

**Behavior:**
* Generates SPARQL query using `sparql.get_object_sparql(mRID)`
* Executes query and parses results
* Creates object instance and adds to graph
* Validates object class against loaded CIM profile

**Usage:**
```python
breaker = db.get_object(mRID='4c04f838-62aa-475e-aefa-a63b7c889c13')
print(breaker)
```

**Source:** `cimgraph/databases/sparql_endpoint.py:163`

#### get_from_triple(subject, predicate, graph=None)

Retrieves the values of a specific attribute for a CIM object.

**Parameters:**
* `subject` (object): The CIM object instance to query
* `predicate` (str): The attribute name (e.g., `'name'`, `'Terminals'`)
* `graph` (Graph): Optional existing graph

**Returns:**
* `list[str] | list[object]`: List of values (strings for primitives, objects for associations)

**Behavior:**
* Adds subject to graph if not already present
* Generates SPARQL query using `sparql.get_triple_sparql(subject, predicate)`
* Parses results using `edge_query_parser()`
* Returns both primitive values and associated objects

**Usage:**
```python
breaker = network.first(cim.Breaker)
terminals = db.get_from_triple(breaker, 'Terminals')
name = db.get_from_triple(breaker, 'name')
```

**Source:** `cimgraph/databases/sparql_endpoint.py:196`

### Graph Creation

#### create_new_graph(container, graph=None)

Creates the base typed property graph for a CIM EquipmentContainer.

**Parameters:**
* `container` (object): The CIM container object (typically `Feeder` or `Substation`)
* `graph` (dict): Optional existing graph (creates new if `None`)

**Returns:**
* `Graph`: The populated typed property graph with Equipment, ConnectivityNodes, and Terminals

**Behavior:**
* Adds container to graph
* Generates SPARQL query using `sparql.get_all_nodes_from_container(container)`
* Query retrieves all Equipment, Terminals, ConnectivityNodes, and optionally Measurements
* Creates base topology without detailed attributes (UUIDs only)
* Parses results using `parse_node_query()`

**Usage:**
```python
feeder = cim.Feeder(mRID='49AD8E07-3BF9-A4E2-CB8F-C3722F837B62')
graph = db.create_new_graph(container=feeder)
```

**Note:** This method is typically called internally by `FeederModel` or `GraphModel` subclasses.

**Source:** `cimgraph/databases/sparql_endpoint.py:220`

#### create_distributed_graph(area, graph=None)

Creates a distributed graph for a SubSchedulingArea.

**Parameters:**
* `area` (object): The `SubSchedulingArea` object defining the distributed region
* `graph` (dict): Optional existing graph

**Returns:**
* `Graph`: The populated graph for the distributed area

**Behavior:**
* Validates that `area` is a `SubSchedulingArea` instance
* Adds area to graph
* Generates SPARQL query using `sparql.get_all_nodes_from_area(area)`
* Retrieves all equipment within the scheduling area
* Parses results using `parse_node_query()`

**Usage:**
```python
area = cim.SubSchedulingArea(mRID='area-uuid')
graph = db.create_distributed_graph(area=area)
```

**Source:** `cimgraph/databases/sparql_endpoint.py:244`

#### build_graph_from_list(graph, mrid_list)

Builds a graph from an explicit list of equipment mRIDs.

**Parameters:**
* `graph` (Graph): Existing graph structure to populate
* `mrid_list` (list[str]): List of mRIDs for equipment to include

**Returns:**
* `Graph`: Updated graph with specified equipment

**Behavior:**
* Processes mRIDs in batches of 100 for performance
* Generates SPARQL query using `sparql.get_all_nodes_from_list(eq_mrids, namespace)`
* Retrieves Equipment, Terminals, and ConnectivityNodes for specified mRIDs
* Parses results using `parse_node_query()`

**Usage:**
```python
equipment_list = ['mrid-1', 'mrid-2', 'mrid-3']
graph = db.build_graph_from_list(graph, equipment_list)
```

**Note:** Originally designed for GridAPPS-D Topology Processor integration.

**Source:** `cimgraph/databases/sparql_endpoint.py:268`

### Graph Expansion

#### get_all_edges(graph, cim_class)

Expands the knowledge graph by retrieving all attributes and associations for a CIM class.

**Parameters:**
* `graph` (Graph): The typed property graph containing objects to expand
* `cim_class` (type): The CIM class type to expand (e.g., `cim.Breaker`)

**Returns:** None (modifies graph in-place)

**Behavior:**
* Retrieves all instances of `cim_class` from graph
* Processes in parallel batches of 100 UUIDs
* For each batch:
  - Generates SPARQL query using `sparql.get_all_edges_sparql(graph, cim_class, eq_mrids)`
  - Executes query and parses results with `edge_query_parser()`
  - Creates new objects for associated classes and adds to graph
* Uses `ThreadPoolExecutor` with `os.cpu_count()` workers for parallel execution

**Usage:**
```python
# Expand all breakers in the graph
network.get_all_edges(cim.Breaker)

# Now breakers have all attributes populated
breaker = network.first(cim.Breaker)
print(breaker.name, breaker.ratedCurrent)
```

**Performance:** For 1000 objects, expect ~5-10 queries with parallel execution.

**Source:** `cimgraph/databases/sparql_endpoint.py:444`

#### get_all_attributes(graph, cim_class)

Retrieves all attributes for a CIM class without expanding associations.

**Parameters:**
* `graph` (Graph): The typed property graph containing objects
* `cim_class` (type): The CIM class type to query

**Returns:** None (modifies graph in-place)

**Behavior:**
* Similar to `get_all_edges()` but with `expand_graph=False`
* Retrieves primitive attributes (name, voltage, etc.)
* Associations stored as string URIs instead of creating new objects
* Processes in batches of 100 for performance
* Uses `sparql.get_all_attributes_sparql()` for query generation

**Usage:**
```python
# Get attributes without expanding edges
network.get_all_attributes(cim.ACLineSegment)

# Lines now have name, length, etc. but associated objects are strings
line = network.first(cim.ACLineSegment)
print(line.name, line.length)
```

**Use Case:** Useful when you need object properties but don't want to expand the entire graph.

**Source:** `cimgraph/databases/sparql_endpoint.py:471`

#### get_edges_query(graph, cim_class)

Generates the SPARQL query string for edge retrieval (debugging utility).

**Parameters:**
* `graph` (Graph): The typed property graph
* `cim_class` (type): The CIM class type

**Returns:**
* `str`: The SPARQL query string

**Behavior:**
* Generates query for first 100 objects of `cim_class`
* Returns query string without executing
* Useful for debugging and understanding query structure

**Usage:**
```python
query = db.get_edges_query(network.graph, cim.Breaker)
print(query)
```

**Source:** `cimgraph/databases/sparql_endpoint.py:427`

### Query Parsing

#### parse_node_query(graph, query_output)

Parses SPARQL query results to build the base network topology.

**Parameters:**
* `graph` (dict): Graph structure to update
* `query_output` (dict): SPARQL JSON results from node query

**Returns:**
* `Graph`: Updated graph with parsed nodes

**Behavior:**
* Iterates through query result bindings
* Parses JSON-LD strings for Equipment and Measurements
* Extracts `@id` and `@type` from JSON-LD
* Creates Equipment, Terminal, and ConnectivityNode objects
* Establishes bidirectional associations:
  - Equipment ↔ Terminal
  - ConnectivityNode ↔ Terminal
* Optionally parses Measurement associations
* Validates classes against CIM profile

**Source:** `cimgraph/databases/sparql_endpoint.py:294`

#### edge_query_parser(query_output, graph, cim_class, expand_graph=True)

Parses SPARQL edge query results to populate object attributes and associations.

**Parameters:**
* `query_output` (QueryResponse): SPARQL JSON results from edge query
* `graph` (Graph): Graph structure to update
* `cim_class` (type): The CIM class being expanded
* `expand_graph` (bool): Whether to create new objects for associations (default: `True`)

**Returns:**
* `list[object]`: List of newly created edges/values

**Behavior:**
* Iterates through query result bindings
* For each result:
  1. **Check for association (edge):** If JSON-LD edge present:
     - Parses edge class and mRID
     - If `expand_graph=True`: creates new object via `create_edge()`
     - If `expand_graph=False`: stores as string via `create_value()`
  2. **Check for enumeration:** If value contains namespace URI:
     - Extracts enum class and value (e.g., `PhaseCode.ABC`)
     - Creates enum instance and sets on object
  3. **Handle primitive value:** Otherwise:
     - Calls `create_value()` for type conversion
     - Sets primitive attribute (string, int, float, bool)
* Returns list of all created edges for reference

**Source:** `cimgraph/databases/sparql_endpoint.py:350`

### Data Upload

#### upload(graph)

Uploads a typed property graph to the database.

**Parameters:**
* `graph` (Graph): The typed property graph to upload

**Returns:** None

**Behavior:**
* Iterates through all classes in graph
* For each object:
  - Generates SPARQL INSERT triples using `sparql.upload_triples_sparql(obj)`
  - Executes update via `update()` method
* Uploads all object attributes and associations as RDF triples
* Preserves object types, UUIDs, and relationships

**Usage:**
```python
# Modify graph
breaker = network.first(cim.Breaker)
breaker.name = 'Modified Breaker'

# Upload changes
db.upload(network.graph)
```

**Performance:** Large graphs may take significant time to upload.

**Source:** `cimgraph/databases/sparql_endpoint.py:493`

----

## Common SPARQL Query Generation

All SPARQL endpoints share a common query generation module (`cimgraph.queries.sparql`) that provides:

* `get_object_sparql(mRID)` - Query for single object by mRID
* `get_triple_sparql(subject, predicate)` - Query for specific triple
* `get_all_nodes_from_container(container)` - Query for all topology nodes in a container
* `get_all_nodes_from_area(area)` - Query for distributed graph nodes
* `get_all_nodes_from_list(mrid_list, namespace)` - Query for specific equipment list
* `get_all_edges_sparql(graph, cim_class, eq_mrids)` - Query for object edges and attributes
* `get_all_attributes_sparql(graph, cim_class, eq_mrids)` - Query for attributes only
* `upload_triples_sparql(obj)` - Generate INSERT statement for object

These query generators ensure consistent SPARQL syntax across all database backends.

----

## UML Sequence Diagrams

The following diagrams illustrate the interaction patterns for common operations across SPARQL endpoints.

### Creating a New Graph from Database

In [ ]:
diagram_text = """%%{init: {"theme":"base"}}%%
sequenceDiagram
    actor User
    participant FeederModel
    participant SPARQLEndpoint
    participant QueryModule
    participant Database
    
    note right of User: Load feeder from database
    User ->>+ FeederModel: FeederModel(container, connection)
    FeederModel ->>+ SPARQLEndpoint: create_new_graph(container)
    SPARQLEndpoint ->> SPARQLEndpoint: add_to_graph(container)
    
    SPARQLEndpoint ->>+ QueryModule: get_all_nodes_from_container(container)
    QueryModule -->>- SPARQLEndpoint: SPARQL query string
    
    SPARQLEndpoint ->>+ SPARQLEndpoint: execute(query)
    SPARQLEndpoint ->>+ Database: _execute_raw_query(query)
    Database -->>- SPARQLEndpoint: query results (JSON)
    SPARQLEndpoint -->>- SPARQLEndpoint: QueryResponse
    
    SPARQLEndpoint ->>+ SPARQLEndpoint: parse_node_query(graph, results)
    loop for each result
        SPARQLEndpoint ->> SPARQLEndpoint: parse Equipment JSON-LD
        SPARQLEndpoint ->> SPARQLEndpoint: create_object(Equipment)
        SPARQLEndpoint ->> SPARQLEndpoint: create_object(Terminal)
        SPARQLEndpoint ->> SPARQLEndpoint: create_object(ConnectivityNode)
        SPARQLEndpoint ->> SPARQLEndpoint: link Equipment ↔ Terminal ↔ Node
    end
    SPARQLEndpoint -->>- SPARQLEndpoint: updated graph
    
    SPARQLEndpoint -->>- FeederModel: Graph with base topology
    FeederModel -->>- User: FeederModel instance
"""

Mermaid(diagram_text)

### Expanding Graph with get_all_edges()

In [ ]:
diagram_text = """%%{init: {"theme":"base"}}%%
sequenceDiagram
    actor User
    participant FeederModel
    participant SPARQLEndpoint
    participant QueryModule
    participant Database
    
    note right of User: Expand breaker attributes
    User ->>+ FeederModel: get_all_edges(cim.Breaker)
    FeederModel ->>+ SPARQLEndpoint: get_all_edges(graph, cim.Breaker)
    
    SPARQLEndpoint ->> SPARQLEndpoint: get UUIDs from graph[cim.Breaker]
    SPARQLEndpoint ->> SPARQLEndpoint: split into batches of 100
    
    par Parallel Batch Processing
        loop for each batch
            SPARQLEndpoint ->>+ QueryModule: get_all_edges_sparql(graph, cim.Breaker, batch)
            QueryModule -->>- SPARQLEndpoint: SPARQL query
            
            SPARQLEndpoint ->>+ Database: execute(query)
            Database -->>- SPARQLEndpoint: query results
            
            SPARQLEndpoint ->>+ SPARQLEndpoint: edge_query_parser(results)
            loop for each result
                alt Association (edge)
                    SPARQLEndpoint ->> SPARQLEndpoint: create_edge() to linked object
                else Enumeration
                    SPARQLEndpoint ->> SPARQLEndpoint: create enum instance
                else Primitive value
                    SPARQLEndpoint ->> SPARQLEndpoint: create_value() with type conversion
                end
            end
            SPARQLEndpoint -->>- SPARQLEndpoint: batch complete
        end
    end
    
    SPARQLEndpoint -->>- FeederModel: None (graph updated)
    FeederModel -->>- User: None
"""

Mermaid(diagram_text)

----

## Database Comparison

The following table compares key characteristics of each database backend:

| Feature | XMLFile | Blazegraph | GraphDB | RDFlib | Neo4j |
|---------|---------|------------|---------|--------|-------|
| **Query Language** | XPath | SPARQL | SPARQL | SPARQL | Cypher |
| **Storage Type** | File | Triple Store | Triple Store | Triple Store | Property Graph |
| **Installation** | None | Java | Java/Docker | Python | Java/Docker |
| **Best For** | Small models | Large models | Enterprise | Testing | Graph algorithms |
| **Performance** | Slow (2 min for 8500 nodes) | Fast | Fast | Medium | Fast |
| **Memory Usage** | High | Medium | Medium | High | Medium |
| **Reasoning** | No | No | Yes | Yes | No |
| **ACID** | No | Yes | Yes | Yes | Yes |
| **Distributed** | No | Yes | Yes | No | Yes |
| **Web Interface** | No | Yes | Yes | No | Yes |
| **License** | Open Source | Apache 2.0 | Commercial/Free | BSD | Commercial/Community |

### Recommended Use Cases

* **XMLFile**: Quick testing with IEEE 13/123-bus models, no database setup
* **Blazegraph**: Production use with distribution models up to 10,000 nodes
* **GraphDB**: Enterprise deployments requiring reasoning and data governance
* **RDFlib**: Python-native applications, testing, temporary storage
* **Neo4j**: Advanced graph analytics, path finding, community detection

----

## Environment Variables

All database connections use environment variables for configuration. See [Environment Variables](3_2_env_variables.ipynb) for complete documentation.

### Common Variables

```python
import os

# CIM profile selection (required)
os.environ['CIMG_CIM_PROFILE'] = 'cimhub_2023'

# CIM namespace (optional, auto-detected from profile)
os.environ['CIMG_NAMESPACE'] = 'http://cim.ucaiug.io/CIM101/draft#'

# IEC 61970-301 version (optional, default: 7)
os.environ['CIMG_IEC61970_301'] = '8'

# Validation log level (optional, default: WARNING)
os.environ['CIMG_VALIDATION_LOG_LEVEL'] = 'INFO'
```

### Database-Specific Variables

**SPARQL Endpoints (Blazegraph, GraphDB):**
```python
os.environ['CIMG_URL'] = 'http://localhost:8889/bigdata/namespace/kb/sparql'
```

**Neo4j:**
```python
os.environ['CIMG_HOST'] = 'localhost'
os.environ['CIMG_PORT'] = '7687'
os.environ['CIMG_DATABASE'] = 'neo4j'
os.environ['CIMG_USERNAME'] = 'neo4j'
os.environ['CIMG_PASSWORD'] = 'password'
```

**MySQL:**
```python
os.environ['CIMG_HOST'] = 'localhost'
os.environ['CIMG_PORT'] = '3306'
os.environ['CIMG_DATABASE'] = 'cim_models'
os.environ['CIMG_USERNAME'] = 'root'
os.environ['CIMG_PASSWORD'] = 'password'
```

----

## Quick Start Examples

### Example 1: XML File (No Database)

In [ ]:
import os
os.environ['CIMG_CIM_PROFILE'] = 'cimhub_2023'
import cimgraph.data_profile.cimhub_2023 as cim

from cimgraph.databases import XMLFile
from cimgraph.models import FeederModel

# Load from XML
file = XMLFile(filename='../../sample_models/ieee13.xml')
network = FeederModel(container=cim.Feeder(), connection=file)

print(f"Loaded {len(network.graph[cim.ACLineSegment])} lines")

### Example 2: Blazegraph (SPARQL Endpoint)

In [ ]:
import os
os.environ['CIMG_CIM_PROFILE'] = 'cimhub_2023'
os.environ['CIMG_URL'] = 'http://localhost:8889/bigdata/namespace/kb/sparql'
import cimgraph.data_profile.cimhub_2023 as cim

from cimgraph.databases import BlazegraphConnection
from cimgraph.models import FeederModel

# Connect to Blazegraph
db = BlazegraphConnection()

# Load feeder from database
feeder = cim.Feeder(mRID='49AD8E07-3BF9-A4E2-CB8F-C3722F837B62')
network = FeederModel(container=feeder, connection=db)

# Expand breakers
network.get_all_edges(cim.Breaker)

print(f"Loaded {len(network.graph[cim.Breaker])} breakers")

### Example 3: Switching Databases

In [ ]:
# Same code works with any database backend!
from cimgraph.databases import GraphDBConnection, RDFlibConnection

# Option 1: GraphDB
# db = GraphDBConnection()

# Option 2: RDFlib
# db = RDFlibConnection()

# Option 3: Blazegraph
db = BlazegraphConnection()

# Everything else stays the same
feeder = cim.Feeder(mRID='49AD8E07-3BF9-A4E2-CB8F-C3722F837B62')
network = FeederModel(container=feeder, connection=db)

----

## Additional Documentation

For detailed information on each database backend:

* [Environment Variables Configuration](3_2_env_variables.ipynb)
* [Blazegraph Database](3_3_blazegraph.ipynb)
* [Neo4j Database](3_4_neo4j.ipynb)
* [GraphDB Database](3_5_graphdb.ipynb)
* [MySQL Database](3_6_mysql.ipynb)
* [GridAPPS-D Integration](3_7_gridappsd.ipynb)
* [XML File Parser](3_8_xml_file_parser.ipynb)
* [JSON-LD File Parser](3_9_jsonld_file_parser.ipynb)
* [RDFlib Parser](3_10_rdflib_parser.ipynb)

For usage examples:
* [FeederModel Usage](../04_graph_models/4_2_feeder_model.ipynb)
* [GraphModel API](../04_graph_models/4_1_graph_models.ipynb)